In [53]:
using CSV, DataFrames, StatsBase

In [47]:
df = CSV.read("localization.csv", header=false)

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /Users/xronos/.julia/packages/CSV/OM6FO/src/CSV.jl:40


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,-64,-56,-61,-66,-71,-82,-81,1
2,-68,-57,-61,-65,-71,-85,-85,1
3,-63,-60,-60,-67,-76,-85,-84,1
4,-61,-60,-68,-62,-77,-90,-80,1
5,-63,-65,-60,-63,-77,-81,-87,1
6,-64,-55,-63,-66,-76,-88,-83,1
7,-65,-61,-65,-67,-69,-87,-84,1
8,-61,-63,-58,-66,-74,-87,-82,1
9,-65,-60,-59,-63,-76,-86,-82,1


In [48]:
df[:RegressionTarget] = Matrix(df[[1, 4, 6, 7]]) * [5, 10, 15, 20] + 0.01*randn(2000)

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[48]:1
└ @ Core In[48]:1
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[48]:1
└ @ Core In[48]:1


2000-element Array{Float64,1}:
 -3829.9979080040703
 -3964.9890816077314
 -3940.004605436743
 -3874.9929794174927
 -3900.0050550638493
 -3960.010134475105
 -3979.9985369219976
 -3909.9979774125964
 -3884.986172468431
 -4100.019915758299
 -4069.988543717358
 -3905.0079808945056
 -3904.9822393634877
     ⋮
 -3815.0153625425623
 -3655.0016119384964
 -3780.006282070347
 -3719.9943661015855
 -3970.00487155771
 -4010.00504834897
 -3999.9817748082974
 -4125.007861805372
 -4019.9922962258843
 -4025.014463848183
 -3969.998083798988
 -3954.990293520874

In [52]:
X = StatsBase.standardize(ZScoreTransform, map(Float64, Matrix(df[1:7])), dims=2)

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[52]:1
└ @ Core In[52]:1


2000×7 Array{Float64,2}:
 0.478077   1.28936   0.782308   0.275256    -0.231795   -1.34731   -1.2459
 0.207303   1.20495   0.84217    0.479389    -0.0647823  -1.33451   -1.33451
 0.708444   0.98395   0.98395    0.341103    -0.485416   -1.31193   -1.2201
 0.884579   0.971791  0.274095   0.797367    -0.510813   -1.64457   -0.772449
 0.739762   0.551459  1.02222    0.739762    -0.578359   -0.954965  -1.51987
 0.563639   1.31916   0.647586   0.395747    -0.443716   -1.45107   -1.03134
 0.605834   1.00033   0.605834   0.408586     0.211337   -1.5639    -1.26802
 0.823905   0.643676  1.09425    0.373332    -0.347585   -1.51908   -1.0685
 0.465976   0.919009  1.00962    0.647189    -0.530695   -1.43676   -1.07433
 0.918708   1.08152   0.59309    0.430281    -0.546573   -1.035     -1.44202
 0.489995   1.01768   0.929733   0.489995    -0.389483   -0.917169  -1.62075
 0.50721    1.09895   0.901707   0.309962    -0.183159   -1.5639    -1.07078
 0.598345   1.14466   0.78045    0.41624     -0.31218

UndefVarError: UndefVarError: kmeans not defined